# Import Modules

In [1]:
# Dependencies
import pandas as pd
import sqlalchemy
#auto map module used for mapping sql tables
from sqlalchemy.ext.automap import automap_base
#session command is used to create a session to the database
from sqlalchemy.orm import Session
#create_engine allows us to connect to the sql server, func is used to do mathmatical functions
from sqlalchemy import create_engine, func

# Connect To Database

In [2]:
# Create a Connection String For Our Database
#remember to delete password before turning in
connection_string = "admin22:12345@localhost:5432/saccrewproject2"
engine = create_engine(f'postgresql://{connection_string}')
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# We can view all of the tables that automap found
Base.classes.keys()

['walmartdata']

In [3]:
# We can view all of the tables that automap found
Base.classes.keys()

['walmartdata']

In [5]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Finding Common Data

In [6]:
# Checking if there are any UIDs that match in both tables
#the syntax in sql alchemy to do an inner join is filter and ==
session.query(func.count(walmartdata.brand)).\
    filter(walmart2019.uniqid==walmart2020.brand)[0]

NameError: name 'walmart2019' is not defined

In [20]:
# Now that we found that we have 1415 products that exist in both tables
# We will perform an inner join to get the prices from 2019 and 2020

yearly_data=session.query(
              walmart2019.productname,
              walmart2019.category,
              walmart2019.listprice,
              walmart2020.listprice ).\
              filter(walmart2019.uniqid==walmart2020.uniqid)

In [22]:
# We will then convert the query results to dataframe
yearly_df=pd.DataFrame(yearly_data, columns=['Product_Name',
                                             'Product_Category',
                                             '2019_Price',
                                             '2020_Price'])

# Finding the Products with the largest Price Change

In [9]:
# Subtract the 2020 list price from the 2019 price to get the price difference
# Filter Out the Results with no change
# Order the results by the biggest change
# Select the 10 biggest

bigestpricechange=session.query(
              walmart2019.productname,
              walmart2019.category,
              walmart2019.listprice,
              walmart2020.listprice,
              walmart2020.listprice - walmart2019.listprice).\
              filter(walmart2019.uniqid==walmart2020.uniqid,
                     (walmart2020.listprice - walmart2019.listprice) > 0).\
              order_by((walmart2020.listprice - walmart2019.listprice).desc()).limit(10)

In [21]:
#Display In a Data Frame
pd.DataFrame(bigestpricechange, columns=['Product_Name',
                                         'Product_Category',
                                         '2019_Price',
                                         '2020_Price',
                                         'Price_Change'])


,Product_Name,Product_Category,2019_Price,2020_Price,Price_Change
0,Fury Eclipse Steel-Glass Portable Basketball S...,Sports & Outdoors | Sports | Basketball | Port...,3452.83,3572.90,120.07
1,PEN & POULTRY CHICKEN & ROOST INSECTICIDE SPRA...,Household Essentials | Pest Control | Insect S...,19.38,134.02,114.64
2,"Step2 Great Outdoors Playhouse, with Built-In ...",Toys | Outdoor Play | Kids Playhouses | Playho...,199.99,310.93,110.94
3,Detecto Detecto Eye Level Physician Scale,Health | Home Health Care | Bathroom Scales | ...,100.00,199.04,99.04
4,SSN NB0215CGP 2 Row 15 ft. Powder Coated Prefe...,Sports & Outdoors | Sports | Softball Gear & E...,1582.94,1640.31,57.37
5,Cutter & Buck DryTec Franklin Striped Performa...,Sports & Outdoors | Sports | Golf Equipment | ...,23.99,79.99,56.00
6,LILLEbaby Airflow Baby Carrier - Mist,Baby | Baby Activities & Gear | Baby Carriers ...,73.00,124.99,51.99
7,Dahon Ios D9 Obsidian Folding Bike Bicycle Black,Sports & Outdoors | Bikes | Adult Bikes | Fold...,950.00,999.00,49.00
8,Baby Sleeping Bag Full Sleeve Newborn Sleep Sa...,Baby | Nursery & Decor | Baby Bedding | Swaddl...,0.00,45.02,45.02
9,Bownet 6 x 6 Foot Professional Pop Up Soccer P...,Sports & Outdoors | Sports | Lacrosse | Lacros...,493.99,535.99,42.00


# Aggregate the Data By Category

In [11]:
category_data=session.query(
               walmart2019.category,
               func.count(walmart2019.category),
               func.count(walmart2020.category),
               func.avg(walmart2019.listprice),
               func.avg(walmart2020.listprice),
               func.min(walmart2019.listprice),
               func.min(walmart2020.listprice),
               func.max(walmart2019.listprice),
               func.max(walmart2020.listprice)).\
              filter(walmart2019.uniqid==walmart2020.uniqid).\
              group_by(walmart2019.category)

In [12]:
# Display as a DataFrame
pd.DataFrame(category_data, columns=['Product_Category',
                                         'Number_of_Products_2019',
                                         'Number_of_Prodicts_2020',
                                         'Avg_Price_2019',
                                         'Avg_Price_2020',
                                         'Lowest_Priced_Item_2019',
                                         'Lowest_Priced_Item_2020',
                                         'Highest_Priced_Item_2019',
                                         'Highest_Priced_Item_2020'
                                        ])

,Product_Category,Number_of_Products_2019,Number_of_Prodicts_2020,Avg_Price_2019,Avg_Price_2020,Lowest_Priced_Item_2019,Lowest_Priced_Item_2020,Highest_Priced_Item_2019,Highest_Priced_Item_2020
0,Personal Care | Bath & Body | Hand Lotions & C...,3,3,15.026667,14.336667,11.46,11.54,18.32,18.32
1,Toys | Outdoor Play | Swimming Pools & Spas | ...,1,1,48.190000,47.000000,48.19,47.00,48.19,47.00
2,Sports & Outdoors | Exercise & Fitness | Sport...,3,3,69.793333,69.793333,37.00,37.00,102.69,102.69
3,Health | Diabetes Care | Blood Glucose Test St...,1,1,51.090000,51.090000,51.09,51.09,51.09,51.09
4,Sports & Outdoors | Sports | Field Equipment &...,1,1,90.340000,77.720000,90.34,77.72,90.34,77.72
...,...,...,...,...,...,...,...,...,...
710,Health | Diabetes Care Brands | Accu-Chek | Ac...,1,1,199.990000,199.990000,199.99,199.99,199.99,199.99
711,Sports & Outdoors | Bikes | Bike Components | ...,2,2,62.950000,62.950000,53.95,53.95,71.95,71.95
712,Toys | Outdoor Play | Swimming Pools & Spas | ...,1,1,302.300000,302.300000,302.30,302.30,302.30,302.30
713,Personal Care | Bath & Body | Body Oils,4,4,17.482500,17.927500,9.53,9.53,34.43,34.43
